In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import kwant
import kwant.continuum
import semicon

import adaptive
import holoviews as hv
from holoviews import opts
adaptive.notebook_extension()
from concurrent.futures import ProcessPoolExecutor
from operator import itemgetter

import  sympy
from sympy.physics.matrices import msigma, Matrix
from sympy import eye
from sympy.physics.quantum import TensorProduct

from sympy.utilities.exceptions import SymPyDeprecationWarning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=SymPyDeprecationWarning)

import scipy

hbar=scipy.constants.hbar
me=scipy.constants.m_e
e=scipy.constants.e

use_semicon = False

ham_metal = ("(-mu_m + V(x, y, z)) * kron(sigma_z, sigma_0, sigma_0) + "
             "C_m * (k_x**2 + k_y**2 + k_z**2) * kron(sigma_z, sigma_0, sigma_0)")

ham_3DTI = ("- mu_ti * kron(sigma_z, sigma_0, sigma_0) + "
            "{epsilon} * kron(sigma_z, sigma_0, sigma_0) + "
            "{M} * kron(sigma_z, sigma_0, sigma_z) - "
            "A_perp * k_x * kron(sigma_z, sigma_y, sigma_x) + "
            "A_perp * k_y * kron(sigma_0, sigma_x, sigma_x) + "
            "A_z * k_z * kron(sigma_z, sigma_0, sigma_y)")

epsilon = "(C_0 - C_perp * (k_x**2 + k_y**2) - C_z * k_z**2)"
M = "(M_0 - M_perp * (k_x**2 + k_y**2) - M_z * k_z**2)"
ham_3DTI = ham_3DTI.format(epsilon=epsilon, M=M)
ham_3DTI = ham_3DTI.format(C_0="C_0")

SC_complex = ("- re({Delta}) * kron(sigma_y, sigma_y, sigma_0) -"
              "im({Delta}) * kron(sigma_x, sigma_y, sigma_0)")

mag = "+ m_z * kron(sigma_z, sigma_z, sigma_0)"

SC_final = SC_complex.format(Delta="Deltaf(x, y, z)")
SC_final_lead = SC_complex.format(Delta="Deltaf_lead(x, y, z)")

term_imp = " + S_imp(site, Smag_imp) * kron(sigma_z, sigma_0, sigma_0)" # to be added to model Hamiltonian in disordered region

ham_3DTI_SC = ham_3DTI + SC_final + mag + term_imp # 3D TI BdG Hamiltonian with superconducting pairing potential
ham_3DTI_SC_lead = ham_3DTI + SC_final_lead + mag

ham_3DTI_SC_discr, coords = kwant.continuum.discretize_symbolic(ham_3DTI_SC)
ham_3DTI_SC_discr_lead, coords = kwant.continuum.discretize_symbolic(ham_3DTI_SC_lead)
ham_metal_discr, coords = kwant.continuum.discretize_symbolic(ham_metal)

### This is where the peierls phase is applied!!!
signs = np.array([-1, -1, -1, -1, 1, 1, 1, 1]) # negative charge for the particles, positive charge for the holes

if use_semicon:
    signs = [-1, -1, -1, -1, 1, 1, 1, 1]
    vector_potential='[-B_z * y, -B_x * z, -B_y * x]'
    ham_3DTI_SC_discr = semicon.peierls.apply(ham_3DTI_SC_discr, coords, A=vector_potential, signs=signs)

a = 10
temp_ti = kwant.continuum.build_discretized(ham_3DTI_SC_discr, coords, grid=a)
temp_ti_lead = kwant.continuum.build_discretized(ham_3DTI_SC_discr_lead, coords, grid=a)
temp_metal = kwant.continuum.build_discretized(ham_metal_discr, coords, grid=a)

ModuleNotFoundError: No module named 'semicon'

In [ ]:
# shape of system
def get_shape(L, W, T, V_x):
    L_start, W_start, T_start = 0, 0, 0
    L_stop, W_stop, T_stop = L, W, T

    def shape(site):
        (x, y, z) = site.pos - V_x
        return (W_start <= y <= W_stop and
                T_start <= z <= T_stop and
                L_start <= x <= L_stop)

    return shape, np.array(V_x)+np.array([L_stop, W_stop, T_stop])

In [ ]:
L_x_ti = 50
L_x_metal = 100
W_y = 100
T_z = 100

syst = kwant.Builder()

_ = syst.fill(temp_metal, *get_shape(L_x_metal, W_y, T_z, (L_x_ti + a , 0, 0)))
_ = syst.fill(temp_ti, *get_shape(L_x_ti, W_y, T_z, (0, 0, 0)))


In [ ]:
sigma_0 = np.identity(2)
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1j], [1j, 0]])
sigma_z = np.array([[1, 0], [0, -1]])

conservation_law = -np.kron(sigma_z, np.kron(sigma_0, sigma_0))


lead_0 = kwant.Builder(kwant.TranslationalSymmetry((-a, 0, 0)))
lead_1 = kwant.Builder(kwant.TranslationalSymmetry((a, 0, 0)), conservation_law=conservation_law)

lead_0.fill(temp_ti_lead, *get_shape(L_x_ti + L_x_metal, W_y, T_z, (0, 0, 0)))
lead_1.fill(temp_metal, *get_shape(L_x_ti + L_x_metal, W_y, T_z, (0, 0, 0)))


syst.attach_lead(lead_0)
syst.attach_lead(lead_1)

In [ ]:
kwant.plotter.set_engine("plotly")
fig = kwant.plot(syst, show=False)
fig.update_layout(scene_aspectmode='data')
fig.show()

In [ ]:
if use_semicon:
    fsyst = syst.finalized()
else:
    fsyst, phase = kwant.builder.add_peierls_phase(syst, signs=signs)

In [ ]:
toy_A = dict(A_perp=3, A_z=3, M_0=0.3, M_perp=15, M_z=15, C_0=0, C_perp=0, C_z=0)
f = dict(
    re=lambda x: x.real,
    im=lambda x: x.imag,
    phi_0=1.0, # units with flux quantum equal to 1
    exp=np.exp
)

In [ ]:
def conductance(lead_in, lead_out, energy, params):
    smatrix = kwant.smatrix(fsyst, energy=energy, params=params)
    
    c_input = smatrix.submatrix((lead_in, 0), (lead_in, 0)).shape[0]
    c_ee = smatrix.transmission((lead_out, 0), (lead_in, 0))
    c_eh = smatrix.transmission((lead_out, 1), (lead_in, 0))

    if lead_in == lead_out:
        c = c_input - c_ee + c_eh
    else:
        c = c_ee - c_eh
        
    return c

def normal_conductance(lead_in, lead_out, energy, params):
    smatrix = kwant.smatrix(fsyst, energy=energy, params=params)
    c_ee = smatrix.transmission((lead_out, 0), (lead_in, 0))
    
    return c_ee

def tunnel_conductance_wrapper(xy):
    Smag_imp, energy = xy
    V = 0.006
    #energy = 0
    B_x = 0.5
    mu_ti = 0.06
    delta = 0.0002
    #Smag_imp = 0
    mu_m = 0.025

    
    def get_S_imp(salt):
        def S_imp(site, Smag_imp):
            return 2 * Smag_imp * (kwant.digest.uniform(site.tag, salt='imp'+salt) - 0.5)
        return S_imp
    
    params = dict(
        **f,
        **toy_A,
        C_m=(hbar**2)/(2*13*me*e*1e-20),
        V=lambda x, y, z: V if 60 <= x <= 80 else 0,
        mu_ti=mu_ti,
        mu_m=mu_m,
        m_z=0,
        Deltaf=lambda x, y, z: 0,
        #Deltaf_lead=lambda x, y, z: delta if z > 50 else 0,
        Deltaf_lead=lambda x, y, z: delta*np.exp(1j*np.angle(z + 1j * y)),
        Smag_imp = Smag_imp,
        S_imp = get_S_imp("abc")
    )
    
    if use_semicon:
        params.update(dict(B_x = B_x/(W_y*T_z), B_y = 0, B_z = 0))
    else:
        B = [B_x/(0.5*W_y*T_z), 0 , 0]
        params.update(phase(B, B, B))
    
    data = dict(c = conductance(1, 1, energy, params))
    
    return data

def normal_conductance_wrapper(xy):
    V, mu_m = xy
    energy = 0
    B_x = 0.5
    mu_ti = 0.02
    delta = 0.0002
    
    params = dict(
        **f,
        **toy_A,
        C_m=(hbar**2)/(2*13*me*e*1e-20),
        V=lambda x, y, z: V if 60 <= x <= 80 else 0,
        #V=lambda x, y, z: 0,
        mu_ti=mu_ti,
        mu_m=mu_m,
        m_z=0,
        Deltaf=lambda x, y, z: 0,
        Deltaf_lead=lambda x, y, z: delta if z > 50 else 0,
    )
    
    if use_semicon:
        params.update(dict(B_x = B_x/(W_y*T_z), B_y = 0, B_z = 0))
    else:
        B = [B_x/(0.5*W_y*T_z), 0 , 0]
        params.update(phase(B, B, B))
        
    smatrix = kwant.smatrix(fsyst, energy=energy, params=params)
    
    data = dict(
        t = smatrix.transmission((0, 0), (1, 0)),
        r = smatrix.transmission((1, 0), (1, 0))
    )
    
    return data

In [ ]:
V_bounds = [0, 0.01]
Smag_imp_bounds = [0, 0.16]
energy_bounds = [-2.5e-4, 2.5e-4]
mu_ti_bounds = [0, 0.08]
mu_metal_bounds = [0, 0.05]
B_x_bounds = [0, 1]

_learner = adaptive.Learner2D(tunnel_conductance_wrapper, bounds=[Smag_imp_bounds, energy_bounds])
learner = adaptive.DataSaver(_learner, arg_picker=itemgetter('c'))
fname = 'conductance_metal_ti_junktion.pickle'
#learner.load(fname)

In [ ]:
executor1 = ProcessPoolExecutor(max_workers=11)
runner = adaptive.Runner(learner, executor=executor1, goal=lambda l: l.loss() < 0.00005)
runner.start_periodic_saving(save_kwargs=dict(fname=fname), interval=60)

In [ ]:
runner.live_info()

In [ ]:
#runner.task.result()

In [ ]:
def get_plotter(title='', xlabel='x', ylabel='y', clabel='G (e²/h)'):
    def plotter(learner):
        plot = learner.plot(tri_alpha=0.2)
        imge = plot.Image.I.opts(colorbar=True, frame_height=400, frame_width=800, title=title, xlabel=xlabel, ylabel=ylabel, clabel=clabel)
        paths = plot.EdgePaths.I
        
        hlines = (hv.HLine(-2e-4) * hv.HLine(2e-4)).opts(opts.HLine(color='red', line_width=1, line_dash='dashed'))
        
        return imge * paths * hlines
    return plotter

In [ ]:
dims2 = dict(
    z=hv.Dimension(('G', '(e²/h)'), unit='(e²/h)', range=(0, 4)),
)



title = 'BTK (vortex):B_x=0.5, mu_ti=0.06, mu_m=0.025, V=6e-3, delta=2e-4, '

runner.live_plot(update_interval=1, plotter=get_plotter(title=title, xlabel='Smag_imp', ylabel='E')) # .redim(**dims2)

In [ ]:
import numpy as np
import scipy.sparse.linalg as sl

def calc_energies(syst, num_states):
    B_x = 0.5
    mu_ti = 0.0005
    mu_m = 0.0005
    Smag_imp = 0
    
    def get_S_imp(salt):
        def S_imp(site, Smag_imp):
            return 2 * Smag_imp * (kwant.digest.uniform(site.tag, salt='imp'+salt) - 0.5)
        return S_imp
    
    params = dict(
        **f,
        **toy_A,
        C_m=(hbar**2)/(2*13*me*e*1e-20),
        #V=lambda x, y, z: V if 50 <= x <= 60 else 0,
        V=lambda x, y, z: 0,
        mu_ti=mu_ti,
        mu_m=mu_m,
        m_z=0,
        Deltaf=lambda x, y, z: 0,
        Deltaf_lead=lambda x, y, z: 0,
        Smag_imp = Smag_imp,
        S_imp = get_S_imp("abc")
    )
    
    if use_semicon:
        params.update(dict(B_x = B_x/2*(W*H), B_y = 0, B_z = 0))
    else:
        B = [B_x/(W_y*T_z), 0 , 0]
        params.update(phase(B, B, B))
    
    ham = syst.hamiltonian_submatrix(params=params, sparse=True).tocsc()
    print(ham.dtype)
    energies, states = sl.eigsh(ham, sigma=0, k=num_states)

    return energies, states

In [ ]:
energies, states = calc_energies(fsyst, num_states=10)

In [ ]:
import plotly.graph_objects as go

rho = kwant.operator.Density(fsyst)
density = rho(states[:, 0])
density /= np.max(density, axis=0, keepdims=True)

In [ ]:
fig = go.Figure(
    data=[
        go.Volume(
            x=[site.pos[0] for site in fsyst.sites],
            y=[site.pos[1] for site in fsyst.sites],
            z=[site.pos[2] for site in fsyst.sites],
            value=density,
            #isomin=0.1,00348664
            #isomax=0.8,
            opacity=0.1, # needs to be small to see through all surfaces
            surface_count=17, # needs to be a large number for good volume rendering
        )
    ]
)
fig.update_layout(margin=dict(l=1, r=1, t=0, b=0), scene_aspectmode='data')
fig.show()